In [130]:
import datetime
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup


URl_MAIN = 'https://www.nsi.bg'

# Bulgarian National Statistics Institute
BG_NSI_URL = {
    'main': 'https://www.nsi.bg',
    'pages': {
        'life_expectancy':
            '/bg/content/3018/смъртност-и-средна-продължителност-на-предстоящия-живот-на-населението-по-местоживеене',
            'mortality_per_week': '/bg/node/18121/',
    },
    'files': {
        'mortality_per_week': '/bg/node/18121/',
        'life_expectancy': '/sites/default/files/files/data/timeseries/Pop_3.1_tab_mortality_DR.xls',
    }
}


In [140]:
url = BG_NSI_URL['main'] + BG_NSI_URL['pages']['mortality_per_week']

req = get(url)
page = BeautifulSoup(req.content, 'lxml')
mortality_per_week_file = page.find(lambda tag: tag.name == 'a' and 'Умирания в България по седмици в периода' in tag.text)
mortality_per_week_path = mortality_per_week_file['href']
file_req = get(BG_NSI_URL['main']+mortality_per_week_path)
date = datetime.date.today()

df = pd.read_excel(file_req.content, engine='openpyxl')

df.dropna(axis=1, how='all', inplace=True)
headers = df.iloc[2]
df.dropna(axis=0, how='any', inplace=True)
df.columns = headers
df.rename(columns={ df.columns[0]: "Year" },inplace=True)
df.replace('x', np.nan, inplace = True)

avg_years_start = df[df['Year'].str.startswith('2015')].index.values[0]
avg_years_end = df[df['Year'].str.startswith('2019')].index.values[0]
analized_year = df[df['Year'].str.startswith('2020')].index.values[0]

excess_death = pd.DataFrame()
excess_death['average_death'] = df.loc[avg_years_start:avg_years_end].mean()
death_2020 = df.loc[analized_year]
excess_death['death_2020'] = death_2020
excess_death = excess_death[excess_death['death_2020']!='-']
excess_death['excess_death'] = excess_death['death_2020'] - excess_death['average_death']

excess_death.reset_index(inplace=True)
excess_death.rename(columns={  excess_death.columns[0]: "Week" },inplace=True)


In [142]:
excess_death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Week           53 non-null     object 
 1   average_death  53 non-null     float64
 2   death_2020     53 non-null     object 
 3   excess_death   53 non-null     object 
dtypes: float64(1), object(3)
memory usage: 1.8+ KB


,Week,average_death,death_2020,excess_death
0,W1,2552.6,2068,-484.6
1,W2,2541.8,2239,-302.8
2,W3,2495.8,2148,-347.8
3,W4,2479.0,2349,-130.0
4,W5,2429.6,2403,-26.6
5,W6,2283.8,2310,26.2
6,W7,2263.4,2271,7.6
7,W8,2254.2,2129,-125.2
8,W9,2248.0,2149,-99.0
9,W10,2236.2,2017,-219.2
